In [0]:
%run /Users/josephbabaloladavies@gmail.com/Clean_Data_Batch

# Imports

In [0]:
from pyspark.sql.functions import col, count, row_number, percentile_approx, avg, format_number, max, rank, year
from pyspark.sql.window import Window


# Task 4
Find the most popular Pinterest category people post to based on their country.


Your query should return a DataFrame that contains the following columns:

country
category
category_count, a new column containing the desired query output

In [0]:
# Join the "pin" and "geo" DataFrames on the "ind" column
df_pin_geo = df_pin.join(df_geo, df_pin["ind"] == df_geo["ind"], how="inner")

# Select relevant columns for the analysis
df_pin_geo_country_category = df_pin_geo.select("country", "category")

# Create a window specification to partition by "country" and order by "category_count" in descending order
windowCountryByCatCount = Window.partitionBy("country").orderBy(col("category_count").desc())

# Perform the analysis to find the most popular category in each country
query_df = df_pin_geo_country_category.groupBy("country", "category").agg(count("category").alias("category_count")) \
    .withColumn("rank", row_number().over(windowCountryByCatCount)) \
    .filter(col("rank") == 1) \
    .drop("rank")
    
display(query_df)

country,category,category_count
Albania,mens-fashion,26
Algeria,travel,4
American Samoa,travel,3
Angola,diy-and-crafts,10
Anguilla,home-decor,22
Antarctica (the territory South of 60 deg S),christmas,9
Antigua and Barbuda,quotes,4
Argentina,tattoos,3
Armenia,diy-and-crafts,33
Aruba,tattoos,24



# Task 5
Find how many posts each category had between 2018 and 2022.


Your query should return a DataFrame that contains the following columns:

post_year, a new column that contains only the year from the timestamp column
category
category_count, a new column containing the desired query output

In [0]:
# Select relevant columns for the analysis
df_pin_geo_country_timestamp = df_pin_geo.select("category", "timestamp")

# Extract the "year" from the "timestamp" column
df_pin_geo_country_timestamp = df_pin_geo_country_timestamp.withColumn("year", year(col("timestamp")))

# Perform the analysis to count the number of posts for each category in each year
query_df_2 = df_pin_geo_country_timestamp.groupBy("year", "category").agg(count("category").alias("category_count")) \
    .filter(col("year") <= 2022) \
    .filter(col("year") >= 2018)

# Rename the "year" column to "post_year" for clarity
query_df_2 = query_df_2.withColumnRenamed("year", "post_year")

# Order the result by "post_year" in descending order
query_df_2 = query_df_2.orderBy(col("post_year").desc())

display(query_df_2)

post_year,category,category_count
2022,vehicles,24
2022,beauty,7
2022,christmas,12
2022,tattoos,6
2022,mens-fashion,4
2021,art,4
2021,event-planning,24
2021,tattoos,10
2021,home-decor,22
2021,quotes,4



# Task 6
Step 1: For each country find the user with the most followers.


Your query should return a DataFrame that contains the following columns:

country
poster_name
follower_count
Step 2: Based on the above query, find the country with the user with most followers.


Your query should return a DataFrame that contains the following columns:

country
follower_count
This DataFrame should have only one entry.

In [0]:
# Select relevant columns for the analysis
df_pin_geo_poster_follower = df_pin_geo.select("country", "poster_name", "follower_count")

# Define a window specification partitioned by "country" and ordered by "follower_count" in descending order
windowCountryByFollower = Window.partitionBy("country").orderBy(col("follower_count").desc())

# Find the poster with the highest follower count in each country
query_df_3 = df_pin_geo_poster_follower.withColumn("rank", row_number().over(windowCountryByFollower)) \
    .filter(col("rank") == 1) \
    .drop("rank")

# Define a window specification ordered by "follower_count" in descending order
windowCountryByFollower_2 = Window.orderBy(col("follower_count").desc())

# Find the global top poster with the highest follower count
query_df_4 = query_df_3.withColumn("rank", rank().over(windowCountryByFollower_2)) \
    .filter(col("rank") == 1) \
    .drop("rank") \
    .drop("max(follower_count)") \
    .drop("poster_name")

display(query_df_3)
display(query_df_4)

country,poster_name,follower_count
Albania,Projects with Kids,20000
Algeria,Fun Life Crisis,130000
American Samoa,TheFab20s | Travel+Food+DIY+Listicles,42000
Angola,CraftGossip.com,502000
Anguilla,"Kristen | Lifestyle, Mom Tips & Teacher Stuff Blog",92000
Antarctica (the territory South of 60 deg S),HikenDip,500000
Antigua and Barbuda,"Sumcoco | Decor Ideas, Hairstyles, Nails Fashion Advice",306000
Argentina,Next Luxury,800000
Armenia,Michelle {CraftyMorning.com},892000
Aruba,TheTrendSpotter,211000


country,follower_count
Armenia,892000


# Task 7
What is the most popular category people post to based on the following age groups:

18-24
25-35
36-50
+50
Your query should return a DataFrame that contains the following columns:

age_group, a new column based on the original age column
category
category_count, a new column containing the desired query output

In [0]:
# Join the "df_pin" and "df_user" DataFrames on the "ind" column
df_pin_user = df_pin.join(df_user, df_pin.ind == df_user.ind)

# Select relevant columns for the analysis
df_pin_user_age_cat = df_pin_user.select("age", "category")

# Create a new column "age_group" based on age ranges
df_pin_user_age_cat = df_pin_user_age_cat.withColumn(
    "age_group",
    when((col("age") >= 18) & (col("age") <= 24), "18-24") \
    .when((col("age") >= 25) & (col("age") <= 35), "25-35") \
    .when((col("age") >= 36) & (col("age") <= 50), "36-50") \
    .otherwise("+50")
)

# Define a window specification partitioned by "age_group" and ordered by "category_count" in descending order
windowAgeGroupByCatCount = Window.partitionBy("age_group").orderBy(col("category_count").desc())

# Find the most popular category in each age group
query_df_5 = df_pin_user_age_cat.groupBy("age_group", "category") \
    .agg(count("category").alias("category_count")) \
    .withColumn("rank", row_number().over(windowAgeGroupByCatCount)) \
    .filter(col("rank") == 1) \
    .drop("rank")

display(query_df_5)

age_group,category,category_count
+50,event-planning,4
18-24,mens-fashion,51
25-35,diy-and-crafts,48
36-50,diy-and-crafts,26


#Task 8
What is the median follower count for users in the following age groups:

18-24
25-35
36-50
+50
Your query should return a DataFrame that contains the following columns:

age_group, a new column based on the original age column
median_follower_count, a new column containing the desired query output
Getting stuck on this tas

In [0]:
# Select relevant columns for the analysis
df_pin_user_age_follower = df_pin_user.select("age", "follower_count")

# Create a new column "age_group" based on age ranges
df_pin_user_age_follower = df_pin_user_age_follower.withColumn(
    "age_group",
    when((col("age") >= 18) & (col("age") <= 24), "18-24") \
    .when((col("age") >= 25) & (col("age") <= 35), "25-35") \
    .when((col("age") >= 36) & (col("age") <= 50), "36-50") \
    .otherwise("+50")
)

# Calculate the median follower count for each age group
query_df_6 = df_pin_user_age_follower.groupBy("age_group") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))

# Format the "median_follower_count" column to display with zero decimal places
query_df_6 = query_df_6.withColumn("median_follower_count", format_number("median_follower_count", 0))

display(query_df_6)

age_group,median_follower_count
18-24,"30,000"
25-35,"43,000"
36-50,"6,000"
+50,111


#Task 9
Find how many users have joined between 2015 and 2020.


Your query should return a DataFrame that contains the following columns:

post_year, a new column that contains only the year from the timestamp column
number_users_joined, a new column containing the desired query output

In [0]:
# Select relevant columns for the analysis
df_pin_user_post_timestamp = df_user.select("date_joined", "user_name", "ind")

# Extract the year from the "date_joined" column
df_pin_user_post_timestamp = df_pin_user_post_timestamp.withColumn("date_joined", year(col("date_joined")))

# Remove duplicate rows based on the "ind" column
df_pin_user_post_timestamp = df_pin_user_post_timestamp.dropDuplicates()

# Group by the "date_joined" column and calculate the count of users joined in each year
query_df_7 = df_pin_user_post_timestamp.groupBy("date_joined") \
    .agg(count("date_joined").alias("number_users_joined")) \
    .filter(col("date_joined") >= 2015) \
    .filter(col("date_joined") <= 2020)

# Rename the "date_joined" column to "post_year"
query_df_7 = query_df_7.withColumnRenamed("date_joined", "post_year")

display(query_df_7)

post_year,number_users_joined
2015,29
2016,29
2017,7


#Task 10
Find the median follower count of users have joined between 2015 and 2020.


Your query should return a DataFrame that contains the following columns:

post_year, a new column that contains only the year from the timestamp column
median_follower_count, a new column containing the desired query output

In [0]:
# Select relevant columns for the analysis
df_pin_geo_date_joined_follower = df_pin_user.select("date_joined", "follower_count")

# Extract the year from the "date_joined" column
df_pin_geo_date_joined_follower = df_pin_geo_date_joined_follower.withColumn("date_joined", year(col("date_joined")))

# Remove duplicate rows based on the "date_joined" column
df_pin_geo_date_joined_follower = df_pin_geo_date_joined_follower.dropDuplicates()

# Group by the "date_joined" column and calculate the median follower count for each year
df_pin_geo_date_joined_follower = df_pin_geo_date_joined_follower.groupBy("date_joined") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .filter(col("date_joined") >= 2015) \
    .filter(col("date_joined") <= 2020)

# Rename the "date_joined" column to "post_year"
df_pin_geo_date_joined_follower = df_pin_geo_date_joined_follower.withColumnRenamed("date_joined", "post_year")

# Format the "median_follower_count" column to display as an integer
query_df_7 = df_pin_geo_date_joined_follower.withColumn("median_follower_count", format_number("median_follower_count", 0))

display(query_df_7)

post_year,median_follower_count
2015,"51,000"
2016,"25,000"
2017,"5,000"


#Task 11
Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.


Your query should return a DataFrame that contains the following columns:

age_group, a new column based on the original age column
post_year, a new column that contains only the year from the timestamp column
median_follower_count, a new column containing the desired query output

In [0]:
# Select relevant columns for the analysis
df_pin_user_age_year_follower = df_pin_user.select("user_name", "age", "date_joined", "follower_count")

# Extract the year from the "date_joined" column and assign it to the "post_year" column
df_pin_user_age_year_follower = df_pin_user_age_year_follower.withColumn("post_year", year(col("date_joined")))

# Create an "age_group" column based on the specified age ranges
df_pin_user_age_year_follower = df_pin_user_age_year_follower.withColumn(
    "age_group",
    when((col("age") >= 18) & (col("age") <= 24), "18-24")\
    .when((col("age") >= 25) & (col("age") <= 35), "25-35")\
    .when((col("age") >= 36) & (col("age") <= 50), "36-50")\
    .otherwise("+50")
)

# Group by "post_year" and "age_group" columns and calculate the median follower count for each group
query_df_8 = df_pin_user_age_year_follower.groupBy("post_year", "age_group") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))

display(query_df_8)

post_year,age_group,median_follower_count
2015,25-35,51000
2016,18-24,25000
2015,18-24,92000
2017,+50,5000
2016,25-35,43000
2017,36-50,6000
2017,18-24,940
2016,36-50,7000
2016,+50,0
2015,36-50,25000
